In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import os
import sys
import datetime
import numpy as np
import re
import logging
from dateutil.relativedelta import relativedelta, TH
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import talib as ta

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
ohlcFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlcWithDelOI.csv"
yhfinanceFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\correctedEODDataYF.csv"

In [3]:
ohlc = pd.read_csv(ohlcFl)
ohlc["date"] = pd.to_datetime(ohlc.date).apply(lambda x: x.date())
yFin = pd.read_csv(yhfinanceFl)
yFin["Date"] = pd.to_datetime(yFin.Date, format="%Y%m%d").apply(lambda x: x.date())

In [4]:
ohlc.loc[ohlc.symbol == "NSE500", "date"].max()

datetime.date(2022, 3, 25)

In [5]:
yFin.rename(
    columns={
        "Date": "date",
        "Ticker": "symbol",
        "Close": "corrClose",
        "High": "corrHigh",
        "Low": "corrLow",
        "Open": "corrOpen",
        "Volume": "yfVolume",
    },
    inplace=True,
)

In [6]:
finalOHLC = pd.merge(
    left=yFin,
    right=ohlc,
    left_on=["symbol", "date"],
    right_on=["symbol", "date"],
    how="outer",
)

In [7]:
def getNifty500():

    nList = "nifty500"
    url = "https://archives.nseindia.com/content/indices/ind_" + nList + "list.csv"

    return pd.read_csv(url)


nifty500 = getNifty500()

In [8]:
tenYearAgo = datetime.date.today() - relativedelta(years=10)
finalOHLC10yr = finalOHLC.loc[finalOHLC.date >= tenYearAgo, :]

In [9]:
finalOHLC10yrNifty500 = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(nifty500.Symbol), :
].dropna(subset=["corrOpen", "corrClose", "corrHigh", "corrLow"])

In [10]:
finalOHLC10yrNifty500 = finalOHLC10yrNifty500[
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ]
]
finalOHLC10yrNifty500.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
    },
    inplace=True,
)
finalOHLC10yrNifty500.head()

,date,symbol,close,high,low,open,volume,delivery,OI,company,industry
862102,2012-03-26,3MINDIA,3724.000000,3778.000000,3717.000000,3717.550049,168.0,164.0,NaN,3M India Ltd.,CONSUMER GOODS
862103,2012-03-26,AARTIDRUGS,4.221517,4.457236,4.180802,4.242946,21169.0,16724.0,NaN,Aarti Drugs Ltd.,PHARMA
862104,2012-03-26,AARTIIND,12.293584,12.985162,12.097465,12.716788,19043.0,14450.0,NaN,Aarti Industries Ltd.,CHEMICALS
862105,2012-03-26,ABB,767.605286,799.392898,759.658368,799.392898,37712.0,4915.0,383250.0,ABB India Ltd.,INDUSTRIAL MANUFACTURING
862106,2012-03-26,ABBOTINDIA,1372.647339,1400.707282,1364.087483,1398.876292,981.0,668.0,NaN,Abbott India Ltd.,PHARMA


In [11]:
finalOHLC10yrNifty500.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [12]:
finalOHLC10yrIndex = finalOHLC10yr.loc[
    finalOHLC10yr.industry == "INDEX",
    [
        "date",
        "symbol",
        "open",
        "high",
        "low",
        "close",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrIndex.head()

,date,symbol,open,high,low,close,volume,delivery,OI,company,industry
1907850,2012-03-26,NSENIFTY,5274.35,5274.95,5174.90,5184.25,169785835.0,0.0,34610200.0,IND_NSENIFTY,INDEX
1907851,2012-03-27,NSENIFTY,5242.95,5277.95,5184.65,5243.15,208775421.0,0.0,35216650.0,IND_NSENIFTY,INDEX
1907852,2012-03-28,NSENIFTY,5231.70,5236.55,5169.60,5194.75,145803597.0,0.0,NaN,IND_NSENIFTY,INDEX
1907853,2012-03-29,NSENIFTY,5145.95,5194.30,5135.95,5178.85,242557889.0,0.0,40016650.0,IND_NSENIFTY,INDEX
1907854,2012-03-30,NSENIFTY,5206.60,5307.10,5203.65,5295.55,164790056.0,0.0,23018850.0,IND_NSENIFTY,INDEX


In [13]:
finalOHLC10yrIndex.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [14]:
finalOHLC10yrIndex.loc[finalOHLC10yrIndex.symbol == "NSE500", "date"].max()

datetime.date(2022, 3, 25)

In [15]:
othersImp = [
    "CRUDE_OIL",
    "GOLD",
    "HDFC_US_ADR",
    "ICICI_US_ADR",
    "INFOSYS_US_ADR",
    "USDINR",
    "DR_REDDY_US_ADR",
    "SILVER",
    "TATAMOTORS_US_ADR",
    "WIPRO_US_ADR",
]
finalOHLC10yrOther = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(othersImp),
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "yfVolume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrOther["company"] = "OTHERS"
finalOHLC10yrOther["industry"] = "OTHERS"
finalOHLC10yrOther.head()

,date,symbol,corrClose,corrHigh,corrLow,corrOpen,yfVolume,delivery,OI,company,industry
29555,2012-03-26,CRUDE_OIL,107.029999,107.320000,106.190002,106.790001,162276.0,NaN,NaN,OTHERS,OTHERS
29556,2012-03-26,GOLD,1685.500000,1692.900024,1657.800049,1665.000000,84.0,NaN,NaN,OTHERS,OTHERS
29557,2012-03-26,HDFC_US_ADR,15.643509,15.657463,15.378365,15.522565,1140800.0,NaN,NaN,OTHERS,OTHERS
29558,2012-03-26,ICICI_US_ADR,5.605964,5.605964,5.502180,5.514954,10418650.0,NaN,NaN,OTHERS,OTHERS
29559,2012-03-26,INFOSYS_US_ADR,5.227291,5.238217,5.148986,5.173570,10643200.0,NaN,NaN,OTHERS,OTHERS


In [16]:
finalOHLC10yrOther.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [17]:
finalOHLC10yrOther.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
        "yfVolume": "volume",
    },
    inplace=True,
)

In [ ]:
corrDf = pd.DataFrame(columns=["stock", "testStock", "corr"])
for sym in finalOHLC10yrNifty500.symbol.unique():
    industry = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, "industry"
    ].values[0]
    stock = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, ["date", "symbol", "close"]
    ].sort_values(by=["date"])
    stock["ret"] = stock["close"].pct_change()
    stockNumOfDates = stock.date.shape[0]
    for testSym in finalOHLC10yrNifty500.loc[
        (
            (finalOHLC10yrNifty500.industry == industry)
            & (finalOHLC10yrNifty500.symbol != sym)
        ),
        :,
    ]["symbol"].unique():
        testStock = finalOHLC10yrNifty500.loc[
            finalOHLC10yrNifty500.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        noOfDates = testStock.shape[0]
        if stockNumOfDates == noOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    for testSym in finalOHLC10yrIndex.symbol.unique():
        testStock = finalOHLC10yrIndex.loc[
            finalOHLC10yrIndex.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
    for testSym in finalOHLC10yrOther.symbol.unique():
        testStock = finalOHLC10yrOther.loc[
            finalOHLC10yrOther.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    # break

In [19]:
corrDf.sort_values(by=["corr"], ascending=False).to_csv(
    r"/Users/shubhangi/DeepDocuments/Trading/NSEData/corr.csv", index=False
)

NameError: name 'corrDf' is not defined

In [18]:
allData = finalOHLC10yrOther.append(finalOHLC10yrIndex).append(finalOHLC10yrNifty500)
allData["date"] = allData.date.apply(lambda x: x.strftime("%Y%m%d"))
allData.fillna(0, inplace=True)
allData.to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\nifty500OtherOHLC10yrs.csv",
    index=False,
)

In [19]:
allData.loc[allData.symbol == "NSE500", "date"].max()

'20220325'

### Below code is for analysis stock by stock

In [80]:
symbol = "PGHL"

In [81]:
import nsepy as npy

start_date = datetime.date.today() - relativedelta(months=6)
end_date = datetime.date.today()

ohlc_data = npy.get_history(
    symbol=re.sub("&", "%26", symbol), start=start_date, end=end_date
).reset_index()

In [82]:
ohlc_data["vol_per_trade"] = ohlc_data["Volume"] / ohlc_data["Trades"]
ohlc_data.sort_values(by="Date", ascending=False).to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlc_data.csv",
    index=False,
)

In [76]:
ohlc_data.shape

(124, 16)

In [42]:
# symbol = symbol

In [43]:
mypath = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data"
holidays = pd.read_excel(
    mypath + os.path.sep + "NSEData" + os.path.sep + "nse_holidays.xlsx"
)
holidays.holidays = holidays.holidays.apply(lambda x: x.date())
# symbols = pd.read_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "FNO.csv")
today = datetime.datetime.today()
from_date = today - relativedelta(months=6)
dtList = list(pd.date_range(from_date, today, freq="B").to_pydatetime())
expriyDf = pd.DataFrame()
oiData = pd.DataFrame()
for dd in dtList:
    if holidays.holidays.isin([datetime.date(dd.year, dd.month, dd.day)]).sum() == 0:
        end_of_month = dd + relativedelta(day=31)
        expiry_date = end_of_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date.year, expiry_date.month, expiry_date.day
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date = expiry_date - relativedelta(day=1)
        end_of_next_month = dd + relativedelta(months=1, day=31)
        expiry_date_next_month = end_of_next_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_next_month.year,
                            expiry_date_next_month.month,
                            expiry_date_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_next_month = expiry_date_next_month - relativedelta(day=1)

        end_of_month_after_next_month = dd + relativedelta(months=2, day=31)
        expiry_date_month_after_next_month = (
            end_of_month_after_next_month + relativedelta(weekday=TH(-1))
        )
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_month_after_next_month.year,
                            expiry_date_month_after_next_month.month,
                            expiry_date_month_after_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_month_after_next_month = (
                    expiry_date_month_after_next_month - relativedelta(day=1)
                )
        if dd > expiry_date:
            expiry_date = expiry_date_next_month
            expiry_date_next_month = expiry_date_month_after_next_month
            end_of_month_after_next_month = dd + relativedelta(months=3, day=31)
            expiry_date_month_after_next_month = (
                end_of_month_after_next_month + relativedelta(weekday=TH(-1))
            )
            while True:
                if (
                    holidays.holidays.isin(
                        [
                            datetime.date(
                                expiry_date_month_after_next_month.year,
                                expiry_date_month_after_next_month.month,
                                expiry_date_month_after_next_month.day,
                            )
                        ]
                    ).sum()
                    == 0
                ):
                    break
                else:
                    expiry_date_month_after_next_month = (
                        expiry_date_month_after_next_month - relativedelta(day=1)
                    )

        expriyDf = expriyDf.append(
            pd.DataFrame(
                {
                    "date": [dd],
                    "current_exp_date": [expiry_date],
                    "next_month_exp_date": [expiry_date_next_month],
                    "month_after_next_month_exp_date": [
                        expiry_date_month_after_next_month
                    ],
                }
            )
        )
for _, row in expriyDf.iterrows():
    logging.info(
        "fetching features data for %s for date %s exp dates %s %s %s"
        % (
            symbol,
            datetime.datetime.strftime(row["date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(row["current_exp_date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(row["next_month_exp_date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(
                row["month_after_next_month_exp_date"].date(), "%Y-%m-%d"
            ),
        )
    )
    current_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["current_exp_date"].date(),
    )
    next_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["next_month_exp_date"].date(),
    )
    month_after_next_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["month_after_next_month_exp_date"].date(),
    )
    if (
        (len(current_month_features["Open Interest"].values) == 1)
        and (len(next_month_features["Open Interest"].values) > 0)
        and (len(month_after_next_month_features["Open Interest"].values) > 0)
    ):
        oiData = oiData.append(
            pd.DataFrame(
                {
                    "Symbol": [symbol],
                    "Date": [row["date"].date()],
                    "cummOI": [
                        current_month_features["Open Interest"].values[0]
                        + next_month_features["Open Interest"].values[0]
                        + month_after_next_month_features["Open Interest"].values[0]
                    ],
                }
            )
        )

In [44]:
ohlc_oi_date = pd.merge(
    left=oiData,
    right=ohlc_data,
    left_on=["Date", "Symbol"],
    right_on=["Date", "Symbol"],
    how="inner",
)

In [45]:
# ohlc_oi_date["vol_per_trade"] = ohlc_oi_date["Volume"] / ohlc_oi_date["Trades"]
ohlc_oi_date.sort_values(by="Date", ascending=False).to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlc_oi_date.csv",
    index=False,
)

In [46]:
ohlc_oi_date.set_index("Date", inplace=True)
ohlc_oi_date.sort_index(inplace=True)

In [47]:
ohlc_oi_date["5 Day avg Del Vol"] = ta.MA(
    ohlc_oi_date["Deliverable Volume"], 5, ta.MA_Type.SMA
).shift(1)

In [48]:
avgDelPct = ohlc_oi_date["%Deliverble"].mean()
avgVolPerTrade = ohlc_oi_date["vol_per_trade"].mean()

In [49]:
ohlc_oi_date["OI Chng"] = ohlc_oi_date["cummOI"] - ohlc_oi_date["cummOI"].shift(1)
ohlc_oi_date["% OI Change"] = (
    ohlc_oi_date["cummOI"] - ohlc_oi_date["cummOI"].shift(1)
) / ohlc_oi_date["cummOI"].shift(1)
ohlc_oi_date["% Price Change"] = (
    ohlc_oi_date["Close"] - ohlc_oi_date["Close"].shift(1)
) / ohlc_oi_date["Close"].shift(1)

In [50]:
def oiAnalysis(prcChng, oiChg):
    if prcChng > 0 and oiChg > 0:
        return "LongBuildUp"
    elif prcChng > 0 and oiChg < 0:
        return "ShortCovering"
    elif prcChng < 0 and oiChg > 0:
        return "ShortBuildUp"
    elif prcChng < 0 and oiChg < 0:
        return "LongUnWinding"


ohlc_oi_date["oiAnalysis"] = ohlc_oi_date.apply(
    lambda x: oiAnalysis(x["% Price Change"], x["% OI Change"]), axis=1
)

In [51]:
ohlc_oi_date.sort_index(ascending=False, inplace=True)

In [52]:
last5Days = 5
LBD_Days = sum(ohlc_oi_date.iloc[0:last5Days, :]["oiAnalysis"] == "LongBuildUp")
cls_higher_vwap = ohlc_oi_date.iloc[0, :]["Close"] > ohlc_oi_date.iloc[0, :]["VWAP"]
del_pct_higher_than_avg = ohlc_oi_date.iloc[0, :]["%Deliverble"] > avgDelPct
vol_per_trade_than_avg = ohlc_oi_date.iloc[0, :]["vol_per_trade"] > avgVolPerTrade

In [53]:
signal = "None"
if (
    del_pct_higher_than_avg
    and vol_per_trade_than_avg
    and cls_higher_vwap
    and LBD_Days > 1
    and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
):
    signal = "VERY_STRONG"
elif (
    del_pct_higher_than_avg
    and vol_per_trade_than_avg
    and cls_higher_vwap
    and LBD_Days == 1
    and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
):
    signal = "STRONG"
else:
    signal = "None"
print(signal)

None


### download FII data

In [52]:
endDate = datetime.date.today()
startDate = endDate - relativedelta(months=6)
dtSrs = pd.Series(pd.date_range(startDate, endDate, freq="SM")).apply(
    lambda x: x.strftime("%b%d%Y")
)
dtSrsLong = pd.Series(pd.date_range(startDate, endDate, freq="SM")).apply(
    lambda x: x.strftime("%B%d%Y")
)
dateDf = pd.DataFrame({"shrtDt": dtSrs, "longDt": dtSrsLong})
data = pd.DataFrame()

for _, dd in dateDf.iterrows():
    urlShrt = (
        f"https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_%s.html"
        % (dd["shrtDt"])
    )
    urlLng = (
        f"https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_%s.html"
        % (dd["longDt"])
    )
    if data.shape[0] == 0:
        try:
            data = pd.read_html(urlShrt)[0].iloc[3:, [1, 32]]
            data.columns = ["Sector", dd["shrtDt"]]

        except:
            data = pd.read_html(urlLng)[0].iloc[3:, [1, 32]]
            data.columns = ["Sector", dd["longDt"]]

    else:
        try:
            temp = pd.read_html(urlShrt)[0].iloc[3:, [1, 32]]
            temp.columns = ["Sector", dd["shrtDt"]]
        except:
            temp = pd.read_html(urlLng)[0].iloc[3:, [1, 32]]
            temp.columns = ["Sector", dd["longDt"]]
        data = pd.merge(
            left=data, right=temp, left_on="Sector", right_on="Sector", how="outer"
        )

data.to_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "fii.csv", index=False)

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

### ANALYZE STOCK

In [19]:
allData.head()

,date,symbol,close,high,low,open,volume,delivery,OI,company,industry
29495,20120319,CRUDE_OIL,108.089996,108.239998,106.550003,107.180000,110353.0,0.0,0.0,OTHERS,OTHERS
29496,20120319,GOLD,1666.900024,1668.000000,1652.099976,1661.199951,462.0,0.0,0.0,OTHERS,OTHERS
29497,20120319,HDFC_US_ADR,15.471396,15.587688,15.257421,15.322544,1394000.0,0.0,0.0,OTHERS,OTHERS
29498,20120319,ICICI_US_ADR,5.856644,5.890175,5.786390,5.810340,8132850.0,0.0,0.0,OTHERS,OTHERS
29499,20120319,INFOSYS_US_ADR,5.290117,5.331090,5.220007,5.283743,11276800.0,0.0,0.0,OTHERS,OTHERS
